In [ ]:
# A default setup cell.
# It imports environment variables, define 'devtools.debug" as a buildins, set PYTHONPATH, and code auto-reload
# Copy it in other Notebooks


from dotenv import load_dotenv
from rich import print

load_dotenv(verbose=True)

%load_ext autoreload
%autoreload 2
%reset -f


In [1]:
import pandas as pd
from smolagents import (
    CodeAgent,
    LiteLLMModel,
)

from src.ai_core.llm import LlmFactory
from src.utils.config_mngr import global_config

2025-04-15 17:49:40.663 | INFO     | src.utils.config_mngr:singleton:99 - selected config=carbon_monitor_local
2025-04-15 17:49:40.683 | INFO     | src.ai_core.cache:set_method:89 - LLM cache : SQLiteCache


In [ ]:
MODEL_ID = None
model_name = LlmFactory(llm_id=MODEL_ID).get_litellm_model_name()
llm = LiteLLMModel(model_id=model_name)
print(model_name)


In [7]:
DATASETS = {
    "country CO2 emissions": (
        "Country daily CO2 emissions from 2019 to 2024",
        "carbon_global_dataset/carbon_global.csv",
    ),
    "cities CO2 emissions": (
        "Major cities daily CO2 emissions from 2019 to 2024",
        "carbon_global_dataset/carbon_cities.csv",
    ),
    "GDP": ("Countries GDP per year", "gdp/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_2.csv"),
    "country population": ("Country populatipn from 1960 to 2023", "world_population/world_population.csv"),
}


def get_data(dataset: str) -> pd.DataFrame:
    """Load and cache CO2 emissions data from configured dataset.

    Returns:
        DataFrame containing emissions data by country and sector.
    """
    description, data_set = DATASETS[dataset]
    data_file = global_config().get_path("datasets_root") / data_set
    print(data_file, description)
    assert data_file.exists(), f"file  not found: {data_file}"
    if data_file.name.endswith(".csv"):
        df = pd.read_csv(data_file, decimal=",")
    else:
        df = pd.read_excel(data_file)
    return df


In [8]:
get_data("country CO2 emissions").head()

/home/tcl/ln_to_onedrive/_ongoing/green_horizon/data/carbon_global_dataset/carbon_global.csv Country daily CO2 emissions from 2019 to 2024


,Unnamed: 0,country,date,sector,value
0,0,Brazil,01/01/2019,Domestic Aviation,0.0237501
1,1,Brazil,01/01/2019,Ground Transport,0.201663
2,2,Brazil,01/01/2019,Industry,0.240156
3,3,Brazil,01/01/2019,International Aviation,0.0183979
4,4,Brazil,01/01/2019,Power,0.107727


In [ ]:
agent = CodeAgent(
    model=llm,
    tools=[get_data_frame],
    additional_authorized_imports=["pandas", "matplotlib.pyplot", "numpy", "json"],
)

agent.run("what was the CO2 emmissions of Brazil for energy generation in 2023")

In [1]:
get_data("country CO2 emissions")

NameError: name 'get_data' is not defined